In [1]:
import copy
import fnmatch
import json
import getpass
import os
import pathlib
import datetime

from os import listdir
from os.path import isfile, join
from distutils import dir_util             
from dask.distributed import LocalCluster, SSHCluster, Client, as_completed  
from laserfarm import Retiler, DataProcessing, GeotiffWriter, MacroPipeline
from laserfarm.remote_utils import get_wdclient, get_info_remote, list_remote

def last_modified(opts, remote_path):
    info = get_info_remote(get_wdclient(opts), remote_path.as_posix())
    format_ = '%a, %d %b %Y %H:%M:%S GMT'
    return datetime.datetime.strptime(info['modified'], format_)

# Macro-Pipeline AHN4 Workflow - Retiling

## Set Run-Specific Input

 Choose whether you want to i) run all input files, ii) run the only input files listed in `filename`, or iii) run the input that was updated since the last workflow run.

In [10]:

remote_path_root = pathlib.Path('/project/lidarac/Data/AHN4')

# path to raw LAZ files 
remote_path_input = remote_path_root / 'splitted'

# path where to copy retiled LAZ files
remote_path_output = remote_path_input.parent / 'retiled'

run = 'from_file' # 'all', 'updated', 'from_file'
filename = 'retile_failed.json'  # if run is 'from_file', set name of file with input file names
assert run in ['all', 'updated', 'from_file']

## Check Connection to Remote Storage

In [ ]:
laz_files = [f for f in listdir(remote_path_input) if isfile(join(remote_path_input, f))
             if f.lower().endswith('.LAZ')]
print('Found: {} LAZ files'.format(len(laz_files)))
if run == 'updated':
    laz_files = [f for f in laz_files]
elif run == 'from_file':
    with open(filename, 'r') as f:
        laz_files_read = json.load(f)
    # check whether all files are available on dCache
    assert all([f in laz_files for f in laz_files_read]), f'Some of the files in {filename} are not in remote dir'
    laz_files = laz_files_read
print('Retrieve and retile: {} LAZ files'.format(len(laz_files)))

## Setup Cluster

Setup Dask cluster used for the macro-pipeline calculation.

In [ ]:
from dask.distributed import Client

client = Client("tcp://10.0.1.12:43843")
client

## Re-tiling

The raw point cloud files are downloaded and retiled to a regular grid.

In [7]:
# details of the retiling schema - for the Netherlands
grid = {
    'min_x': -113107.81,
    'max_x': 398892.19,
    'min_y': 214783.87,
    'max_y': 726783.87,
    'n_tiles_side': 512
}


In [8]:
# set path where output will be written 

retiling_input = {
    'setup_local_fs': {
        'input_folder': remote_path_input.as_posix(),
        'output_folder': remote_path_output.as_posix()
    },
    'set_grid': grid,
    'split_and_redistribute': {},
    'validate': {}
}


In [12]:
macro = MacroPipeline()

# add pipeline list to macro-pipeline object and set the corresponding labels
macro.tasks = [Retiler(file).config(retiling_input) for file in laz_files]
macro.set_labels([os.path.splitext(file)[0] for file in laz_files])

macro.setup_cluster(cluster="tcp://10.0.1.12:43843")

# run! 
macro.run()

# save outcome results and check that no error occurred before continuing
macro.print_outcome(to_file='retile.out')

failed = macro.get_failed_pipelines()
if failed:
    with open('retile_failed.json', 'w') as f:
        json.dump([pip.label + '.LAZ' for pip in failed], f)
    raise RuntimeError('Some of the pipelines have failed')

In [ ]:
failed = macro.get_failed_pipelines()
if failed:
    with open('retile_failed.json', 'w') as f:
        json.dump([pip.label + '.LAZ' for pip in failed], f)
    raise RuntimeError('Some of the pipelines have failed')

## Terminate cluster

In [ ]:
macro.shutdown()

## Troubleshooting 

### Cancel all jobs and restart the notebook

Copy and paste these lines in a separate Python shell. If the Dask dashboard shows that some tasks are still queued to be processed, run the lines again - this should clear the scheduler up and give back control to the current notebook. Normally proceed to terminate the cluster and restart the notebook.

In [ ]:
from dask.distributed import Client, Future
client = Client('tcp://145.100.59.123:8786')
futures = [Future(key) for key in client.who_has().keys()]
client.cancel(futures)

In [ ]:
client.shutdown()
